In [2]:
import requests
from bs4 import BeautifulSoup as Soup
import time
import pandas as pd

headers = {'Accept':'*/*','Accept-Encoding':'gzip, deflate','Accept-Language':'zh-CN,zh;q=0.9','Connection':'keep-alive','User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'          }
count = float(Soup(requests.get('http://zdb.pedaily.cn/inv/',headers = headers).text, "lxml").find('span',class_='total').text)
print(count)

13128.0


In [22]:
from codecs import open 

def save_data(path,data='',mode='a+',encoding='utf-8'):
    with open(path,mode=mode,encoding=encoding) as f:
        f.write(data)

def parse_info(soup):
    info =soup.find('div',class_='info').text
    info = info.replace(u'年','-').replace(u'月','-').replace(u'日','')
    company =soup.find('dt',class_='company').text.replace(u',','，')
    industry=soup.find('dt',class_='industry').text
    #money=soup.find('dt',class_='money').text    
    r=soup.find('span',class_='r')
    r=r.text if r else ''
    d=soup.find('span',class_='d')
    d=d.text if d else ''
    m=soup.find('span',class_='m')
    m=m.text if m else ''    
    group=soup.find('dt',class_='group').text
    return [info,company,industry,r,m,d,group]    

def parse_page(url,path):
    soup = Soup(requests.get(url,headers= headers).text)
    lis = soup.find('ul',id='inv-list').find_all('li')[1:]
    for infos in lis:
        #print(parse_info(infos))
        data = parse_info(infos)
        data.append('\r\n')
        save_data(path,','.join(data))
    
save_path = '/home/git/PycharmProjects/github/jupyter/data/inv.csv'
#parse_page('http://zdb.pedaily.cn/inv/y2017-p1/',save_path)
save_data(save_path,','.join([u'投资时间',u'受资方',u'所属行业',u'VC阶段',u'投资金额',u'币种',u'投资方','\r\n']))
parse_page('http://zdb.pedaily.cn/inv/y2017-p1/',save_path)          

In [23]:
import math
save_data(save_path,','.join([u'投资时间',u'受资方',u'所属行业',u'VC阶段',u'投资金额',u'币种',u'投资方','\r\n']))
for page in range(1,int(math.ceil(count/20))):
    url = 'http://zdb.pedaily.cn/inv/y2017-p%i/' %page
    parse_page(url,save_path)